In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from skimage.feature import graycomatrix, graycoprops
import numpy as np
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)

### Load sample points and sar data


In [ ]:
# Load training points
# shp_path = "../sample_collection/sample_eaton_2.shp"
shp_path = "../sample_collection/eaton_random_sample.shp"


gdf = gpd.read_file(shp_path)

# Load SAR GeoTIFF
input_sar_image_path  = "../SAR_Data_processing_eaton/8_subset/eaton_S1A_IW_GRDH_1SDV_2025_01_21_TC_stack_32611_clip.tif"
glcm_raster_path="../SAR_Data_processing_eaton/9_glcm/eaton_S1A_IW_GRDH_1SDV_2025_01_21_Orb_tnr_Bdr_Cal_Spk_TC_stack_glcm.tif"
thermal_image_path="../ThermalData/04_diff_norm/TRAD_diff_norm.tif"
dnbr_image_path="../S2_Data_Processing/Indices/dNBR_eaton.tif"
#following are the input for the model selection and training

output_file_path="output/sample/training_data_eaton_indices.csv"

# final_columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre"]
# "dTRAD","dNBR",
final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"] # "ΔRVI",'Δvv_vh_ratio',"ΔRDFI" "RBD_VV",
# RCBI: https://www.sciencedirect.com/science/article/abs/pii/S0924271619302242
# final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH", "ΔRVI",'Δvv_vh_ratio',"ΔRDFI"]

# final_columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

output_sar_image_path="output/sar_image/eaton_sar_indices.tif"

add_bands=False
add_indices=True
add_glcm=False
thermal_band=True
add_dnbr=True

In [ ]:
# # Load training points
# shp_path = "../sample_collection/refined_sample.shp"

# gdf = gpd.read_file(shp_path)

# # Load SAR GeoTIFF
# input_sar_image_path  = "../SAR_Data_processing/8_subset/palisades_S1A_IW_GRDH_1SDV_TC_stack_gamma_32611.tif"
# glcm_raster_path="../SAR_Data_processing/9_glcm/palisades_S1A_IW_GRDH_1SDV_TC_stack_gamma_glcm_32611.tif"

# #follwing are the input for the model selection and training
# output_file_path="output/sample/training_data_indices.csv"
# # final_columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre"]
# final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI",'Δvv_vh_ratio',"ΔRDFI"]

# # final_columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

# output_sar_image_path="output/sar_image/palisades_sar_indices.tif"

# add_bands=False
# add_indices=True
# add_glcm=False

In [ ]:
# def probabilistic_quantize(image, levels=64):
#     """
#     Apply probabilistic quantization using k-means clustering.

#     :param image: 2D NumPy array (grayscale image)
#     :param levels: Number of quantization levels
#     :return: Quantized image
#     """
#     # Flatten the image into a 1D array
#     image_flat = image.flatten().reshape(-1, 1)

#     # Perform KMeans clustering to define quantization levels
#     kmeans = KMeans(n_clusters=levels, random_state=0).fit(image_flat)

#     # Assign quantized levels based on the clustering result
#     quantized_image = kmeans.labels_.reshape(image.shape)
    
#     return quantized_image

# def compute_glcm_features(image, distances=[4], angles=[0], window_size=7, quantize_levels=32):
# # def compute_glcm_features(image, distances=[4], angles=[0,45,90,135], window_size=9, quantize_levels=32):
#     """
#     Compute GLCM texture features for a given image with probabilistic quantization.

#     :param image: 2D NumPy array (grayscale image)
#     :param distances: List of pixel distances for GLCM calculation
#     :param angles: List of angles for GLCM calculation
#     :param levels: Number of gray levels (default 256 for 8-bit images)
#     :param window_size: Size of the moving window (must be an odd number)
#     :param quantize_levels: Number of levels for probabilistic quantization
#     :return: Dictionary of texture feature images
#     """
#     # Apply probabilistic quantization to the image
#     quantized_image = probabilistic_quantize(image, quantize_levels)
    
#     # Define texture properties to compute
#     properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    
#     # Pad image for window-based processing
#     pad = window_size // 2
#     padded_image = np.pad(quantized_image, pad, mode='edge')
    
#     # Initialize dictionaries to store results
#     feature_images = {prop: np.zeros_like(image, dtype=np.float32) for prop in properties}
    
#     # Loop over each pixel
#     for i in range(pad, padded_image.shape[0] - pad):
#         for j in range(pad, padded_image.shape[1] - pad):
#             window = padded_image[i - pad:i + pad + 1, j - pad:j + pad + 1]
            
#             # Compute GLCM
#             glcm = graycomatrix(window, distances=distances, angles=angles, levels=quantize_levels, symmetric=True, normed=True)
            
#             # Compute texture properties
#             for prop in properties:
#                 feature_images[prop][i - pad, j - pad] = graycoprops(glcm, prop)[0, 0]
    
#     return feature_images


# with rasterio.open(raster_path) as src:
#     profile = src.profile  # Get metadata
#     band_1 = src.read(1) 
#     glcm_band_1=compute_glcm_features(band_1, distances=[4], angles=[0,45,90,135], window_size=9, quantize_levels=32)
#     print(glcm_band_1)


In [ ]:
"""
Log10 of negative value is 0, thus applicanle in calculating the differnce
"""
from scipy.ndimage import generic_filter

# Open input raster
with rasterio.open(input_sar_image_path) as src:
    profile = src.profile  # Get metadata
    bands = src.read()  # Read all bands

# Define new band names (not directly stored in GeoTIFF, but useful for documentation)
# new_band_names = ["gamma_vh_post", "gamma_vv_post", "gamma_vh_pre", "gamma_vv_pre"]

# Save raster with new names
# profile.update(count=len(new_band_names)+5+glcm_bands_counts)  # Update band count if needed
# profile.update(count=len(new_band_names))  # Update band count if needed
profile.update(count=len(final_columns_names))
print(len(final_columns_names),"len(final_columns_names)")
epsilon = 1e-6
band_index=0
with rasterio.open(output_sar_image_path, "w", **profile) as dst:
    if add_bands:
        for i in range(len(bands)):
            dst.write(bands[i], i + 1)
            dst.set_band_description(i + 1, final_columns_names[i])  # Assign names
            band_index=i+1

    
    if thermal_band:
        with rasterio.open(thermal_image_path) as TRAD_dataset:
            # profile = src.profile  # Get metadata
            trad_band = TRAD_dataset.read()
            band_index=band_index+1
            dst.write(trad_band[0], band_index)
            dst.set_band_description(band_index, "dTRAD")
    
    if add_dnbr:
        with rasterio.open(dnbr_image_path) as dnbr_dataset:
            # profile = src.profile  # Get metadata
            dnbr_band = dnbr_dataset.read()
            band_index=band_index+1
            dst.write(dnbr_band[0], band_index)
            dst.set_band_description(band_index, "dNBR")
    
    if add_indices:
        vh_post,vv_post,vh_pre, vv_pre=bands[0],bands[1],bands[2],bands[3]

        # vh_post,vv_post,vh_pre, vv_pre=(bands[2]+bands[6])/2,(bands[3]+bands[7])/2,(bands[0]+bands[4])/2,(bands[1]+bands[5])/2

        # print(np.min(vh_post),np.max(vh_post))
        band_index=band_index+1
        RBD_VV_band = vv_post - vv_pre
        # X_min,X_max = np.min(RBD_VV_band),np.max(RBD_VV_band)
        # RBD_VV_band = (RBD_VV_band - X_min) / (X_max - X_min)
        dst.write(RBD_VV_band, band_index)
        dst.set_band_description(band_index, "RBD_VV")


        band_index=band_index+1
        RBD_VH_band = vh_post - vh_pre
        # X_min,X_max = np.min(RBD_VH_band),np.max(RBD_VH_band)
        # RBD_VH_band = (RBD_VH_band - X_min) / (X_max - X_min)
        dst.write(RBD_VH_band, band_index)
        dst.set_band_description(band_index, "RBD_VH")

        band_index=band_index+1
        # RBR_VV_band = vv_post/ (vv_pre + epsilon)
        RBR_VV_band = (np.log10(vv_post+epsilon))-(np.log10(vv_pre + epsilon)) #logarithmic ratio
        # RBR_VV_band=np.log(RBR_VV_band + epsilon) 
        # X_min,X_max = np.min(RBR_VV_band),np.max(RBR_VV_band)
        # RBR_VV_band = (RBR_VV_band - X_min) / (X_max - X_min)
        dst.write(RBR_VV_band, band_index)
        dst.set_band_description(band_index, "RBR_VV")
        
        band_index=band_index+1
        # pre_std=np.std(vv_pre) #global
        window_size = 7
        # Apply local standard deviation filter
        pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        RBR_VV_band=(np.log10(vv_post))-(np.log10(vv_pre)) #logarithmic ratio
        dod_RBR_VV= np.abs(RBR_VV_band)/pre_std_local
        dst.write(dod_RBR_VV, band_index)
        dst.set_band_description(band_index, "dod_RBR_VV")


        band_index=band_index+1
        # RBR_VH_band = vh_post / (vh_pre+epsilon)
        RBR_VH_band = (np.log10(vh_post+epsilon)) - (np.log10(vh_pre+epsilon))
        # RBR_VH_band=np.log(RBR_VH_band + epsilon) 
        # X_min,X_max = np.min(RBR_VH_band),np.max(RBR_VH_band)
        # RBR_VH_band = (RBR_VH_band - X_min) / (X_max - X_min)
        dst.write(RBR_VH_band, band_index)
        dst.set_band_description(band_index, "RBR_VH")

        band_index=band_index+1
        # pre_std=np.std(vh_pre)
        window_size = 7
        # Apply local standard deviation filter
        pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        RBR_VH_band=(np.log10(vh_post))-(np.log10(vh_pre))
        dod_RBR_VH= np.abs(RBR_VH_band)/pre_std_local
        dst.write(dod_RBR_VH, band_index)
        dst.set_band_description(band_index, "dod_RBR_VH")

        band_index=band_index+1
        # RVI_post = 4* vh_post/(vv_post+vh_post+epsilon) #4* vh/(vv+vh)
        # RVI_pre = 4* vh_pre/(vh_pre+vv_pre+epsilon) #4* vh/(vv+vh)
        RVI_post = (10*np.log10((4* vh_post) + epsilon))-(10*np.log10(vv_post+vh_post+epsilon)) #4* vh/(vv+vh)
        RVI_pre = (10*np.log10((4* vh_pre)+ epsilon))-(10*np.log10(vh_pre+vv_pre+epsilon)) #4* vh/(vv+vh)
        delta_RVI = RVI_post  - RVI_pre
        # delta_RVI=10*np.log10(delta_RVI+epsilon)
        # delta_RVI = np.log(RVI_post + epsilon) - np.log(RVI_pre + epsilon)
        # X_min, X_max = np.min(delta_RVI), np.max(delta_RVI)
        # delta_RVI = (delta_RVI - X_min) / (X_max - X_min)
        dst.write(delta_RVI, band_index)
        dst.set_band_description(band_index, "ΔRVI")

        band_index=band_index+1
        # vv_vh_ratio_post = vv_post/(vh_post+epsilon) 
        # vv_vh_ratio_pre = vv_pre/(vh_pre+epsilon)
        vv_vh_ratio_post = (10*np.log10(vv_post))-(10*np.log10(vh_post+epsilon))
        vv_vh_ratio_pre = 10*np.log10(vv_pre)-(10*np.log10(vh_pre+epsilon))
        delta_vv_vh_ratio=vv_vh_ratio_post-vv_vh_ratio_pre
        # delta_vv_vh_ratio=10*np.log10(delta_vv_vh_ratio+epsilon)
        # X_min, X_max = np.min(delta_vv_vh_ratio), np.max(delta_vv_vh_ratio)
        # delta_vv_vh_ratio = (delta_vv_vh_ratio - X_min) / (X_max - X_min)
        # delta_vv_vh_ratio=np.log(vv_vh_ratio_post+epsilon)-np.log(vv_vh_ratio_pre+epsilon)
        dst.write(delta_vv_vh_ratio, band_index)
        dst.set_band_description(band_index, "Δ_vv_vh_ratio")

        band_index=band_index+1
        RFDI_pre = (vv_pre - vh_pre) / (vv_pre + vh_pre+epsilon)
        RFDI_post = (vv_post - vh_post) / (vv_post + vh_post+epsilon)
        delta_RFDI = RFDI_post - RFDI_pre
        # X_min, X_max = np.min(delta_RFDI), np.max(delta_RFDI)
        # delta_RFDI = (delta_RFDI - X_min) / (X_max - X_min)
        dst.write(delta_RFDI, band_index)
        dst.set_band_description(band_index, "ΔRDFI")

        band_index=band_index+1
        #Radar convolution burn index
        RCBI=(vv_post-vv_pre)+(vh_post-vh_pre)
        dst.write(RCBI, band_index)
        dst.set_band_description(band_index, "RCBI")
    
    if add_glcm:
        # Open input raster
        with rasterio.open(glcm_raster_path) as glcm_dst:
            # profile = src.profile  # Get metadata
            glcm_bands = glcm_dst.read()  # Read all bands
            # print(len(glcm_bands))

        # glcm_bands_counts=len(glcm_bands)/2

        band_index=band_index+1
        with rasterio.open(glcm_raster_path) as glcm_dataset:
            glcm_bands = glcm_dataset.read()  # Read all bands
            # for i in range(16,20):
            #     difference=glcm_bands[i]-glcm_bands[i+20]
            #     dst.write(difference, band_index)
            #     band_index=band_index+1
            #     break
            for i in range(int(len(glcm_bands)/2)):
                corresponding_index,index,  = i, int(len(bands)/2)+i
                difference=glcm_bands[corresponding_index]-glcm_bands[index]
                X_min, X_max = np.min(difference), np.max(difference)
                difference = (difference - X_min) / (X_max - X_min)
                dst.write(difference+epsilon, band_index)
                band_index=band_index+1
    

12 len(final_columns_names)


In [ ]:
# print(np.log(1e-6))
# print(np.log(1e-5))
# print(np.log(1e-4))
# print(np.log(1e-3))
# print(np.log(1e-2))

# print(np.log(-274.77084))
# 0.0003570663 19.881237
# std_sample = np.std(bands[0])
# print(std_sample)

In [ ]:
# with rasterio.open(glcm_raster_path) as glcm_dataset:
#     # profile = src.profile  # Get metadata
#     bands = glcm_dataset.read()  # Read all bands
#     print(len(bands))
#     for i in range(int(len(bands)/2)):
#         band_index=i
#         corresponding_band_index=int(len(bands)/2)+i
#         print(band_index, corresponding_band_index)
#         print(bands[0])

#         # difference=bands[corresponding_band_index]-bands[band_index]
        


In [ ]:
# dataset=rasterio.open()
with rasterio.open(output_sar_image_path) as dataset:
    print(dataset.count)
    # Get coordinates of training points
    coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]

    # Extract values from all bands
    values = [v for v in dataset.sample(coords)]

    # Convert to DataFrame with proper band names
    df = pd.DataFrame(values, columns=final_columns_names)

    # Merge with training sample points
    gdf = gdf.reset_index(drop=True)
    final_df = pd.concat([gdf, df], axis=1)

    # Save as CSV for ML training
    final_df.to_csv(output_file_path, index=False)

12


In [ ]:
final_df.head(10)

,id,Dnbr1,dTRAD1,class,geometry,dTRAD,dNBR,RBD_VV,RBD_VH,RBR_VV,dod_RBR_VV,RBR_VH,dod_RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI
0,0.0,-0.012932,0.390035,0,POINT (402563.977 3781947.786),0.543791,-0.003007,-0.062598,-0.001094,-0.276601,2.225827,-0.048584,0.390992,2.068958,-2.280205,-0.088081,-0.063691
1,1.0,0.257881,0.329700,1,POINT (401001.224 3788317.096),0.262359,0.116568,0.722317,0.155186,0.385482,1.033317,0.281206,0.753800,-0.802476,1.042758,0.085081,0.877502
2,2.0,-0.018341,0.293110,0,POINT (396536.715 3779106.757),0.298560,0.029506,-0.013624,-0.005019,-0.067081,1.308563,-0.165618,3.230963,-0.857903,0.985365,0.051114,-0.018643
3,3.0,0.045327,0.274815,0,POINT (394725.939 3780181.137),0.263916,-0.017009,-0.373323,0.014979,-0.413191,3.527304,0.224723,1.918464,5.897636,-6.379149,-0.202555,-0.358344
4,4.0,0.015972,0.245232,0,POINT (403621.816 3785897.184),0.239393,-0.015621,0.010044,0.002513,0.140269,9.462468,0.197762,13.343007,0.488805,-0.574889,-0.033800,0.012557
5,5.0,0.004455,0.215648,0,POINT (391215.942 3788010.582),0.232386,0.043302,-0.010458,0.001305,-0.123076,4.242761,0.076489,2.637064,1.659862,-1.995684,-0.128331,-0.009153
6,6.0,0.189714,0.562086,1,POINT (396739.847 3785803.935),0.437135,0.189063,0.000831,0.000166,0.003208,0.043100,0.001029,0.013822,-0.013421,0.021790,0.002374,0.000998
7,7.0,0.355512,0.508758,1,POINT (396734.55 3787246.972),0.472947,-0.005080,3.837574,-0.012472,0.777204,2.578157,-0.031046,0.102987,-7.321940,8.082506,0.309975,3.825102
8,8.0,0.022919,0.337096,0,POINT (400108.795 3779769.036),0.300895,0.022176,0.218083,0.003064,0.499232,19.053085,0.038695,1.476805,-3.846117,4.605400,0.288328,0.221147
9,9.0,0.018437,0.305566,0,POINT (392028.617 3779193.377),0.280265,-0.030385,-0.012500,-0.021021,-0.028517,0.183025,-0.256988,1.649396,-1.927408,2.284706,0.138408,-0.033521


In [ ]:
df.agg(['min', 'max'])

,dTRAD,dNBR,RBD_VV,RBD_VH,RBR_VV,dod_RBR_VV,RBR_VH,dod_RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI
min,0.057610,-0.324802,-2.082357,-0.496476,-0.802765,0.000286,-0.995735,0.000481,-14.197667,-10.068534,-0.863026,-2.183869
max,0.966913,0.770360,6.696298,0.615477,1.953531,330.275116,1.567704,208.438995,8.587059,15.798543,0.904178,6.787219


In [ ]:
print((10 * np.log10(101))-(10 * np.log10(5)))
# # print(10 * np.log10(50))
print(10 * np.log10(101/5))
print((10 * np.log10(101))/(10 * np.log10(5)))
# print(10*np.log10(epsilon))

13.05351369446624
13.053513694466236
2.867535604316286
